In [2]:
import pandas as pd
data = pd.read_csv(r"C:\TTTT\nlp\datasets\n_datarng.csv", sep='^')
data

,Описание,ЗП,Навыки
0,"РБ - международная компания, которая создает у...",30000,"{172, 237}"
1,"Требования: Наличие портфолио, а также готовн...",120000,"{68, 10, 173, 82, 243, 244, 215, 216}"
2,"Обязанности: Погрузка-разгрузка Слесарные,...",32000,"{28, 13}"
3,"Рассматриваем без опыта, предусмотрено обучени...",400000,"{169, 203, 47}"
4,Кафе Семейное в поиске кандидатов на позицию О...,25000,"{0, 147, 227}"
...,...,...,...
21820,Высококачественная онлайн-платформа с увлекате...,3000,"{216, 68}"
21821,В команду по комплексному контентному и маркет...,45000,"{201, 137, 203, 169, 206, 49, 147, 216, 27, 61}"
21822,"Добрый день, уважаемый соискатель! Международн...",150000,"{66, 109}"
21823,Крупный комбинат питания приглашает в столовые...,50000,"{208, 193, 219, 157}"


In [4]:
import nltk
import re

nltk.download("stopwords") # поддерживает удаление стоп-слов
nltk.download('punkt') # делит текст на список предложений
nltk.download('wordnet') # проводит лемматизацию
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luksx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luksx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luksx\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:

russian_stopwords = stopwords.words("russian")
new_text = []
lemmatize = nltk.WordNetLemmatizer()
for i in data['Описание']:
    #удаляем неалфавитные символы
    text = re.sub("[^а-яА-Яa-zA-Z]"," ",i.lower()) 
    # токенизируем слова
    text = nltk.word_tokenize(text, language = "russian")
    # лемматирзируем слова
    text = [lemmatize.lemmatize(word) for word in text if not word in set(russian_stopwords)]
    new_text.append(text)


In [10]:
from gensim.models import Word2Vec
Word2Vec_model = Word2Vec(sentences=new_text, vector_size=600, window=5, min_count=5, workers=6)
Word2Vec_model.save(r"C:\TTTT\nlp\models\Word2Vec.model")

In [1]:
import numpy as np
from gensim.models import Word2Vec

Word2Vec_model = Word2Vec.load(r"C:\TTTT\nlp\models\Word2Vec.model")
def vectorize(sentence):
    words_vecs = [Word2Vec_model.wv[word] for word in sentence if word in Word2Vec_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(600)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)



In [6]:
from ast import literal_eval
skills_binary = []
for i in data['Навыки']:
    binr = [0]*256
    for j in literal_eval(i):
        binr[int(j)-1] = 1
    skills_binary.append(binr)

In [7]:
vectorize_text = []
for i in new_text:
    vectorize_text.append(vectorize(i))


In [8]:
from sklearn.model_selection import train_test_split
text_train, text_test, salary_train, salary_test, skills_train, skills_test = train_test_split(vectorize_text, list(data['ЗП']), skills_binary, test_size = 0.1, random_state = 42)


In [48]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Embedding
from CustomLayer import TransformerEncoderLayer
# Define input layer
input_text = Input(shape=(None,), dtype='float32')  # None allows for variable sequence lengths

num_skills = 256
embedding_dim=64
vocab_size = len(Word2Vec_model.wv)

# Embedding and Encoder
x = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_text)
transformer_block = TransformerEncoderLayer(d_model=embedding_dim, num_heads=8, ff_dim=embedding_dim * 4)
x = transformer_block(x)

# Global average pooling to reduce sequence to a single vector
x = GlobalAveragePooling1D()(x)



# Skill output (transformer decoder setup could be added here for dynamic sequence generation)
# Placeholder for Transformer decoder - in practice, this will be more complex and involve looped decoding
skill_output = Dense(num_skills, activation='sigmoid', name='skill_output')(x)  # This is an illustrative placeholder

# Complete the model
model = Model(inputs=input_text, outputs=skill_output)
model.compile(optimizer='adam',
              loss={'skill_output': 'binary_crossentropy'},
              metrics={'skill_output': 'accuracy'})

# Summary
model.summary()

Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_16 (Embedding)        │ (None, None, 64)       │     2,006,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_layer_12    │ (None, None, 64)       │       166,016 │
│ (TransformerEncoderLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_7      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ skill_output (Dense)            │ (None, 256)            │        16,640 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,189,504 (8.35 MB)

 Trainable params: 2,189,504 (8.35 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.metrics import Mean, BinaryAccuracy, MeanAbsoluteError

loss_object_skills = BinaryCrossentropy()

train_loss = Mean(name='train_loss')
train_skill_accuracy = BinaryAccuracy(name='train_skill_accuracy', )

@tf.function
def train_step(inp, tar_skill):
    with tf.GradientTape() as tape:
        skill_predictions = model(inp, training=True)
        loss_skills = loss_object_skills(tar_skill, skill_predictions)
        total_loss = loss_skills

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(total_loss)
    train_skill_accuracy.update_state(tar_skill, skill_predictions)


In [50]:
optimizer = Adam(learning_rate=0.001)

def train_model(model, dataset, epochs=10):
    for epoch in range(epochs):
        for batch, (inp, tar_skill) in enumerate(dataset):
            train_step(inp, tar_skill)

        print(f'Epoch {epoch + 1}, Loss: {train_loss.result()}, Skill Accuracy: {train_skill_accuracy.result()}')

        # Reset states for the next epoch
        train_loss.reset_state()
        train_skill_accuracy.reset_state()

In [46]:
BATCH_SIZE = 32

tf_batched_data = tf.data.Dataset.from_tensor_slices((text_train, skills_train))

tf_batched_data = tf_batched_data.cache()
tf_batched_data = tf_batched_data.batch(BATCH_SIZE)
tf_batched_data = tf_batched_data.prefetch(tf.data.experimental.AUTOTUNE)

In [51]:
train_model(model, dataset=tf_batched_data, epochs=10)

Epoch 1, Loss: 0.09459435939788818, Skill Accuracy: 0.977707028388977


KeyboardInterrupt: 

In [14]:
model.save(r'C:\TTTT\nlp\models\10ep_1st.keras')

In [37]:
from CustomLayer import TransformerEncoderLayer

model = tf.keras.models.load_model(r'C:\TTTT\nlp\models\10ep_1st.keras')

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_2', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, None], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Embedding', 'config': {'name': 'embedding', 'trainable': True, 'dtype': 'float32', 'input_dim': 31357, 'output_dim': 128, 'embeddings_initializer': {'module': 'keras.initializers', 'class_name': 'RandomUniform', 'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}, 'registered_name': None}, 'embeddings_regu

In [136]:
BATCH_SIZE = 1

tf_batched_t_data = tf.data.Dataset.from_tensor_slices((text_test))

tf_batched_t_data = tf_batched_data.cache()
tf_batched_t_data = tf_batched_data.batch(BATCH_SIZE)
tf_batched_t_data = tf_batched_data.prefetch(tf.data.experimental.AUTOTUNE)

In [137]:

new_model = tf.saved_model.load("my_model")
for batch, (inp) in enumerate(tf_batched_t_data):
    new_model.serve(inp)
    break

TypeError: Binding inputs to tf.function failed due to `cannot compute Pack as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:Pack] name: packed`. Received args: ((<tf.Tensor: shape=(32, 600), dtype=float32, numpy=
array([[ 0.18087652, -0.07245839, -0.03983251, ...,  0.14984016,
         0.00898311, -0.24883187],
       [ 0.22719222, -0.03713135, -0.2485353 , ..., -0.02756927,
         0.02181634, -0.065396  ],
       [ 0.12677844, -0.05828552, -0.14398131, ...,  0.10343078,
        -0.16039167, -0.03875295],
       ...,
       [ 0.22203593, -0.02570179,  0.27622724, ...,  0.2630734 ,
         0.00691799, -0.14133948],
       [ 0.19026355, -0.09119177,  0.07464815, ...,  0.02595977,
         0.03739889, -0.15485984],
       [ 0.30460954, -0.06864084,  0.17503898, ...,  0.07287274,
        -0.05500926, -0.14949042]], dtype=float32)>, <tf.Tensor: shape=(32, 256), dtype=int32, numpy=
array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [